# Notes

`env.run()`

* Repeatedly calls `env.step()` until the given criterion `until` is met.
* If `until` is None, then the method will return when there are no more events to process.
* If `until` is an `Event` class, the step() method will keep executing until the event has been triggered.
* If `until` is a number, it will continue stepping until the environment's time reaches that number.

`env.step()`

* Processes the next event. This involves
    * Popping the next event from the heap `env._queue`.
    * Execute each of the event's callbacks.

`Process(env, generator)`, AKA `env.process(generator)`
* Processes an event yielding generator.
* Initially the only event in `env._queue` is thie `Initialize()` event, which produces a `Process()`. The `Initialize().callbacks` are `[Process._resume].`
* When the `Process._resume` is called, the Process gets the next event from the event yielding generator it was initailized with. If the next event produced by the generator has callbacks, then the the `Process._resume` is appended to the end of them, so that it will try and keep obtaining events from the process generator.

# Setup

In [1]:
import os.path
import sys
    
import simpy

"""
Add parent directory to current path
"""
p = os.path.abspath('..')
if p not in sys.path:
    sys.path.insert(0,p)
    
from call_center_simulator import Lead, generate_lead, interval_lead_generator
from call_center_simulator.model import NaiveLeadScoringModel, PerfectLeadScoringModel

# Set parameters

In [2]:
RANDOM_SEED = 42

LEAD_DIST_PARAMS = {
    'min_patience': 5,
    'max_patience': 15,
    'min_ltv': 0,
    'max_ltv': 1000,
    'mean_talk_time': 10,
}

INTERVAL_PARAMS = {
    'num_leads': 20,
    'lead_creation_interval': 2.5,
}

# Run simulation using naive model

In [8]:
"""
Run simulation using naive lead scoring model
"""
import random


random.seed(RANDOM_SEED)

naive_results = {
    'converted_leads': [],
    'missed_leads': [],
}

# Create environment
env = simpy.Environment()

# Create resource
agents = simpy.PriorityResource(env, capacity=1)

# Create lead scoring model
model = NaiveLeadScoringModel()

# Create interval process generator
intv_process_gen = interval_lead_generator(env, agents, model, naive_results,
                                           INTERVAL_PARAMS, LEAD_DIST_PARAMS)

# Schedule execution of processes
env.process(intv_process_gen)

# Run the simulation
env.run()

# Count total LTV accumulated
converted_ltv = sum([l.ltv for l in naive_results['converted_leads']])
missed_ltv = sum([l.ltv for l in naive_results['missed_leads']])

print('\n')
print('Converted Leads: {}'.format(len(naive_results['converted_leads'])))
print('Missed Leads:    {}'.format(len(naive_results['missed_leads'])))
print('Converted LTV: ${:.2f}'.format(converted_ltv))
print('Missed LTV:    ${:.2f}'.format(missed_ltv))

 0.0000 Lead00: Hesre I am
 0.0000 Lead00: Waited  0.000
 2.5259 Lead00: Converted
 2.5502 	Lead01: Hesre I am
 2.5502 	Lead01: Waited  0.000
 3.4597 	Lead01: Converted
 5.8841 		Lead02: Hesre I am
 5.8841 		Lead02: Waited  0.000
 7.2542 			Lead03: Hesre I am
 7.3215 				Lead04: Hesre I am
 7.9440 					Lead05: Hesre I am
12.0415 						Lead06: Hesre I am
12.9233 		Lead02: Converted
12.9233 			Lead03: Waited  5.669
18.2141 				Lead04: HUNG UP after 10.893
19.9203 							Lead07: Hesre I am
19.9254 					Lead05: HUNG UP after 11.981
20.4074 						Lead06: HUNG UP after  8.366
20.7966 			Lead03: Converted
20.7966 							Lead07: Waited  0.876
24.6217 								Lead08: Hesre I am
26.5426 									Lead09: Hesre I am
30.9637 										Lead10: Hesre I am
33.8800 							Lead07: Converted
33.8800 								Lead08: Waited  9.258
34.0120 											Lead11: Hesre I am
34.2199 												Lead12: Hesre I am
36.4220 										Lead10: HUNG UP after  5.458
36.7443 													Lead13: Hesre I am
37.5207 						

# Run simulation using perfect model

In [9]:
"""
Run simulation using naive lead scoring model
"""
import random


random.seed(RANDOM_SEED)

perfect_results = {
    'converted_leads': [],
    'missed_leads': [],
}

# Create environment
env = simpy.Environment()

# Create resource
agents = simpy.PriorityResource(env, capacity=1)

# Create lead scoring model
model = PerfectLeadScoringModel()

# Create interval process generator
intv_process_gen = interval_lead_generator(env, agents, model, perfect_results,
                                           INTERVAL_PARAMS, LEAD_DIST_PARAMS)

# Schedule execution of processes
env.process(intv_process_gen)

# Run the simulation
env.run()

# Count total LTV accumulated
converted_ltv = sum([l.ltv for l in perfect_results['converted_leads']])
missed_ltv = sum([l.ltv for l in perfect_results['missed_leads']])

print('\n')
print('Converted Leads: {}'.format(len(perfect_results['converted_leads'])))
print('Missed Leads:    {}'.format(len(perfect_results['missed_leads'])))
print('Converted LTV: ${:.2f}'.format(converted_ltv))
print('Missed LTV:    ${:.2f}'.format(missed_ltv))

 0.0000 Lead00: Hesre I am
 0.0000 Lead00: Waited  0.000
 2.5259 Lead00: Converted
 2.5502 	Lead01: Hesre I am
 2.5502 	Lead01: Waited  0.000
 3.4597 	Lead01: Converted
 5.8841 		Lead02: Hesre I am
 5.8841 		Lead02: Waited  0.000
 7.2542 			Lead03: Hesre I am
 7.3215 				Lead04: Hesre I am
 7.9440 					Lead05: Hesre I am
12.0415 						Lead06: Hesre I am
12.9233 		Lead02: Converted
12.9233 				Lead04: Waited  5.602
12.9885 				Lead04: Converted
12.9885 			Lead03: Waited  5.734
19.9203 							Lead07: Hesre I am
19.9254 					Lead05: HUNG UP after 11.981
20.4074 						Lead06: HUNG UP after  8.366
20.8618 			Lead03: Converted
20.8618 							Lead07: Waited  0.942
24.6217 								Lead08: Hesre I am
26.5426 									Lead09: Hesre I am
30.9637 										Lead10: Hesre I am
33.9452 							Lead07: Converted
33.9452 									Lead09: Waited  7.403
34.0120 											Lead11: Hesre I am
34.2199 												Lead12: Hesre I am
36.4220 										Lead10: HUNG UP after  5.458
36.7443 													Lead13: H

# Compare results

In [10]:
"""
Determine which leads were converted by perfect model, but not by naive model.
And vice versa.
"""
perfect_converted_ltvs = [l.ltv for l in perfect_results['converted_leads']]
naive_converted_ltvs = [l.ltv for l in perfect_results['missed_leads']]

print('\nLTVs converted by perfect model but not naive model:')
for ltv in sorted(perfect_converted_ltvs, reverse=True):
    if ltv not in naive_converted_ltvs:
        print('${:.2f}'.format(ltv))

print('\nLTVs converted by naive model but not perfect model:')
for ltv in sorted(naive_converted_ltvs, reverse=True):
    if ltv not in perfect_converted_ltvs:
        print('${:.2f}'.format(ltv))
        
        
"""
Determine which leads were missed by perfect model, but not by naive model.
And vice versa.
"""
perfect_missed_ltvs = [l.ltv for l in perfect_results['missed_leads']]
naive_missed_ltvs = [l.ltv for l in naive_results['missed_leads']]

print('\nLTVs missed by perfect model but not naive model:')
for ltv in sorted(perfect_missed_ltvs, reverse=True):
    if ltv not in naive_missed_ltvs:
        print('${:.2f}'.format(ltv))\
        
print('\nLTVs missed by naive model but not perfect model:')
for ltv in sorted(naive_missed_ltvs, reverse=True):
    if ltv not in perfect_missed_ltvs:
        print('${:.2f}'.format(ltv))


LTVs converted by perfect model but not naive model:
$892.18
$861.71
$809.43
$807.13
$649.88
$648.04
$275.03
$229.05
$218.64
$210.98

LTVs converted by naive model but not perfect model:
$655.44
$556.95
$378.53
$370.18
$340.25
$264.88
$227.90
$163.40
$101.00
$92.75

LTVs missed by perfect model but not naive model:
$378.53
$370.18

LTVs missed by naive model but not perfect model:
$861.71
$809.43
